In [15]:
import pandas as pd
import numpy as np

In [18]:
caliper = pd.read_csv(
    '../data/test/caliper_output.txt', 
    sep='|',
    header=0,
    dtype={
        'displacement': np.float64,
        'delay': np.float64
    },
    parse_dates=['time']
)

scale = pd.read_csv(
    "../data/test/scale_output.txt",
    sep='|',
    header=0,
    dtype={
        'weight': np.float64,
        'unit': str,
        'flag': str
    },
    parse_dates=['time']
)

In [47]:
def clean_raw_data(caliper:pd.DataFrame, scale:pd.DataFrame):
    """ Clean raw data heads and tails for data matching later """
    scale_start_ind = 0
    while (caliper.time < scale.time.iloc[scale_start_ind]).sum() == 0:
        scale_start_ind += 1
    caliper_start_ind = caliper.index[caliper.time < scale.time.iloc[scale_start_ind]].max()

    scale_end_ind = scale.index.max()
    while (caliper.time > scale.time.iloc[scale_end_ind]).sum() == 0:
        scale_end_ind -= 1
    caliper_end_index = caliper.index[caliper.time > scale.time.iloc[scale_end_ind]].min()
    clean_scale = scale.iloc[scale_start_ind:scale_end_ind + 1]
    clean_caliper = caliper.iloc[caliper_start_ind:caliper_end_index + 1]

    return clean_caliper.reset_index(drop=True), clean_scale.reset_index(drop=True)

In [48]:
ccaliper, cscale = clean_raw_data(caliper=caliper, scale=scale)

In [49]:
ccaliper

,time,displacement,delay
0,2024-08-15 16:27:04.921583,0.000,-0.02756
1,2024-08-15 16:27:05.024515,0.000,-0.01111
2,2024-08-15 16:27:05.130989,0.000,0.00828
3,2024-08-15 16:27:05.247804,0.000,0.01749
4,2024-08-15 16:27:05.374175,0.000,0.01686
...,...,...,...
73,2024-08-15 16:27:14.121319,14.970,-0.00369
74,2024-08-15 16:27:14.250930,14.970,-0.00232
75,2024-08-15 16:27:14.369943,14.970,0.00471
76,2024-08-15 16:27:14.495472,14.976,0.00813


In [51]:
cscale

,time,weight,unit,flag
0,2024-08-15 16:27:05.002224,-1.0,LG,S
1,2024-08-15 16:27:05.302247,-1.0,LG,S
2,2024-08-15 16:27:05.603463,-1.0,LG,S
3,2024-08-15 16:27:05.903569,-1.0,LG,S
4,2024-08-15 16:27:06.203539,-3.0,LG,M
5,2024-08-15 16:27:06.504448,-7.0,LG,M
6,2024-08-15 16:27:06.804943,-12.0,LG,M
7,2024-08-15 16:27:07.105788,-16.0,LG,M
8,2024-08-15 16:27:07.404270,-9.0,LG,M
9,2024-08-15 16:27:07.704754,-6.0,LG,M
